In [14]:
from sklearn.svm import SVC
from pennylane import numpy as np
import pennylane as qml
from model_parameters import ModelParameters
from tqdm.notebook import tqdm
import time

# Kernel definition

In [15]:
n_qubits = 8

dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    qml.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

# Training QSVM

In [16]:
model_parameters = ModelParameters()

# Initialize the model
model = SVC(kernel=kernel_matrix, C=1.0) 

x_train, x_test, y_train, y_test = model_parameters.get_htru_2()

for i in tqdm(range(model_parameters.num_runs)):
    balanced_x_train, balance_y_train = model_parameters.sub_select_dataset(x_train, y_train, model_parameters.training_samples, balanced=True)
    balanced_x_test, balance_y_test = model_parameters.sub_select_dataset(x_test, y_test, model_parameters.testing_samples)

    # Train the model and make predictions
    start_training_time = time.time()
    model.fit(balanced_x_train, balance_y_train)
    end_training_time = time.time()

    training_duration = end_training_time - start_training_time

    start_testing_time = time.time()
    y_pred = model.predict(balanced_x_test)
    end_testing_time = time.time()

    test_duration = end_testing_time - start_testing_time

    # Store the scores
    model_parameters.append_score(balance_y_test, y_pred, training_duration, test_duration)


  0%|          | 0/1 [00:00<?, ?it/s]

# Prediction

In [17]:
# Calculate mean and standard deviation for each metric and print the results
for metric, values in model_parameters.scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")

Accuracy: 0.983 ± 0.000
Balanced_accuracy: 0.966 ± 0.000
Recall: 0.946 ± 0.000
Specificity: 0.986 ± 0.000
Precision: 0.875 ± 0.000
Npv: 0.994 ± 0.000
Gmean: 0.966 ± 0.000
Informedness: 0.932 ± 0.000


In [18]:
print(model_parameters.scores)

{'accuracy': [0.9825], 'balanced_accuracy': [0.9660859206313752], 'recall': [0.9459459459459459], 'specificity': [0.9862258953168044], 'precision': [0.875], 'npv': [0.9944444444444445], 'gmean': [0.9658759689845493], 'informedness': [0.9321718412627504]}
